# Propositional logic with exhaustification

## Syntax

Note that the following datatype is based on the `hatt` library.

In [1]:
import Prelude hiding (lookup)

In [2]:
newtype Var = Var Char
    deriving (Eq, Ord)

instance Show Var where
    show (Var v) = [v]

data Expr = Variable Var
          | Negation Expr
          | Exh Expr
          | Conjunction Expr Expr
          | Disjunction Expr Expr
          | Conditional Expr Expr
          | Biconditional Expr Expr
          deriving Eq
          
showBinaryConnective :: (Expr -> String) -> String -> Expr -> Expr -> String
showBinaryConnective show_ symbol exp1 exp2 =
  '(' : show_ exp1 ++ " " ++ symbol ++ " " ++ show_ exp2 ++ ")"

showBC :: String -> Expr -> Expr -> String
showBC = showBinaryConnective show
          
instance Show Expr where
  show (Variable      name)      = show name
  show (Negation      expr)      = '¬' : show expr
  show (Exh expr) = 'O' : show expr
  show (Conjunction   exp1 exp2) = showBC "∧" exp1 exp2
  show (Disjunction   exp1 exp2) = showBC "∨" exp1 exp2
  show (Conditional   exp1 exp2) = showBC "→" exp1 exp2
  show (Biconditional exp1 exp2) = showBC "↔" exp1 exp2

### Incorporating alternatives

In [3]:
binaryConnectives :: [(Expr -> Expr -> Expr)]
binaryConnectives = [Conjunction,Disjunction,Conditional,Biconditional]

class Alt a where
    alt :: a -> [a]

instance Alt Expr where
    alt (Variable v) = [(Variable v)]
    alt (Negation exp) = (alt exp) ++ map Negation (alt exp)
    alt (Conjunction exp1 exp2) = [f' exp1' exp2' | f' <- binaryConnectives, exp1' <- (alt exp1), exp2' <- (alt exp2)] ++ (alt exp1) ++ (alt exp2)
    alt (Disjunction exp1 exp2) = [f' exp1' exp2' | f' <- binaryConnectives, exp1' <- (alt exp1), exp2' <- (alt exp2)] ++ (alt exp1) ++ (alt exp2)
    alt (Conditional exp1 exp2) = [f' exp1' exp2' | f' <- binaryConnectives, exp1' <- (alt exp1), exp2' <- (alt exp2)] ++ (alt exp1) ++ (alt exp2)
    alt (Biconditional exp1 exp2) = [f' exp1' exp2' | f' <- binaryConnectives, exp1' <- (alt exp1), exp2' <- (alt exp2)] ++ (alt exp1) ++ (alt exp2)


In [4]:
alt $ Negation (Variable (Var 'a')) `Disjunction` Variable (Var 'b')

[(a ∧ b),(¬a ∧ b),(a ∨ b),(¬a ∨ b),(a → b),(¬a → b),(a ↔ b),(¬a ↔ b),a,¬a,b]

## Semantics

In [49]:
import Data.Map (Map, fromList, lookup)
import Control.Monad (liftM, liftM2, replicateM)
import Data.Maybe (fromMaybe)
import Data.List (group, sort)

type Mapping = Map Var Bool

interpret :: Expr -> Mapping -> Bool
interpret (Variable      v)         vs = fromMaybe False (lookup v vs)
interpret (Negation      expr)      vs = not $ interpret expr vs
interpret (Conjunction   exp1 exp2) vs = interpret exp1 vs && interpret exp2 vs
interpret (Disjunction   exp1 exp2) vs = interpret exp1 vs || interpret exp2 vs
interpret (Conditional   exp1 exp2) vs = not (interpret exp1 vs) || interpret exp2 vs
interpret (Biconditional exp1 exp2) vs = interpret exp1 vs == interpret exp2 vs

assignments :: Expr -> [Mapping]
assignments expr = let vs = variables expr
                       ps = replicateM (length vs) [True, False]
                   in  map (fromList . zip vs) ps
                   
variables :: Expr -> [Var]
variables expr = let vars_ (Variable      v)     vs = v : vs
                     vars_ (Negation      e)     vs = vars_ e vs
                     vars_ (Conjunction   e1 e2) vs = vars_ e1 vs ++ vars_ e2 vs
                     vars_ (Disjunction   e1 e2) vs = vars_ e1 vs ++ vars_ e2 vs
                     vars_ (Conditional   e1 e2) vs = vars_ e1 vs ++ vars_ e2 vs
                     vars_ (Biconditional e1 e2) vs = vars_ e1 vs ++ vars_ e2 vs
                 in  map head . group . sort $ vars_ expr []
                 
values :: Expr -> [Bool]
values expr = map (interpret expr) (assignments expr)

-- | Determines whether an expression is contradictory.
isContradiction :: Expr -> Bool
isContradiction = not . or . values

-- | Determines whether or not an alternative is excludable
excl :: Expr -> Expr -> Bool
excl exp1 exp2 = (not . isContradiction) (Conjunction exp1 (Negation exp2))

-- | Returns the list of excludable alternatives
exclAlts :: Expr -> [Expr]
exclAlts expr = filter (excl expr) (alt expr)